In [1]:
import gc
import shutil
from glob import glob
from itertools import chain
from pathlib import Path

import memray
import daft
import numpy as np
import timm
import torch
from datasets import load_dataset
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

%load_ext memray

In [2]:
USE_DAFT: bool = True # else use torch dataset

BATCH_SIZE = 32
MODEL_NAME = "vit_small_patch14_reg4_dinov2.lvd142m"
IMAGE_GLOB = None
IMAGES_FOLDER = "./tmp-test-images"

TEST_DATASET = "kvriza8/microscopy_images"
NUM_TEST_IMAGES = 1_000

nice_models = [
"mobilenetv3_large_100",
"vit_small_patch14_reg4_dinov2.lvd142m",
"vit_base_patch14_reg4_dinov2.lvd142m",
"vit_large_patch14_reg4_dinov2.lvd142m",
"aimv2_large_patch14_224.apple_pt_dist"
]

In [3]:
def dl_hf_images(dataset_name: str = "kvriza8/microscopy_images",
                 dir: Path = None,
                 max_images: int = 50,
                 overwrite: bool = True) -> None:

    dataset = load_dataset(dataset_name, split="train", streaming=True)
    if overwrite:
        shutil.rmtree(dir, ignore_errors=True)
        dir.mkdir(parents=True, exist_ok=True)

    for i, img_row in enumerate(tqdm(iter(dataset), total=max_images)):
        if i >= max_images:
            break
        img = img_row["image"]
        img.save(dir / f"{i}.png")

    del dataset
    gc.collect()

    return None

In [4]:

tmp_path = Path(IMAGES_FOLDER)
dl_hf_images(dir=tmp_path, max_images=NUM_TEST_IMAGES)

100%|██████████| 1000/1000 [00:22<00:00, 44.72it/s]


In [5]:

class Embedder:
    """instantiate pretrained timm model to generate embeddings"""
    def __init__(self, model_name: str, device: torch.device = None):
        self.model_name = model_name
        # choose device and dtype
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if self.device.type == "cuda":
            self.dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        else:
            self.dtype = torch.float32

        # Create and prepare the model
        self.model = timm.create_model(self.model_name, pretrained=True, num_classes=0)
        self.model.to(self.device, memory_format=torch.channels_last)
        self.model.eval()
        self.model = torch.compile(self.model, dynamic=True, mode="reduce-overhead")

        # must resolve config to drop unneeded fields
        cfg = timm.data.resolve_data_config(self.model.pretrained_cfg)
        self.transform = timm.data.create_transform(**cfg)

    @torch.inference_mode()
    def embed(self, batch_imgs: torch.Tensor) -> torch.Tensor:
        """set up input and embed it"""
        batch_imgs = batch_imgs.to(self.device, non_blocking=True, memory_format=torch.channels_last)
        if self.device.type == "cuda":
            with torch.amp.autocast("cuda", dtype=self.dtype):
                return self.model(batch_imgs)
        return self.model(batch_imgs)

In [6]:
@daft.udf(return_dtype=daft.DataType.python())
class TransformImageCol:
    """run timm embedder on an image column"""
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.embedder = Embedder(self.model_name)

    def __call__(self, batch_images) -> list:
        return [self.embedder.transform(Image.fromarray(im)) for im in batch_images.to_pylist()]

In [7]:

class ImageListIteratorAsDict(Dataset):
    def __init__(self, filelist: list[Path], transform: callable):
        self.filelist = filelist
        self.transform = transform

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx: int):
        image = Image.open(self.filelist[idx]).convert("RGB")
        if self.transform:
            return {"image_transformed": self.transform(image)}
        # return as dict for easy comparison vs. daft
        else:
            return {"image": [image]}


In [8]:

def get_file_list(source: str | list[str]) -> list[Path]:
    """
    Given a folder path, a glob pattern, or a filelist, return a list of image file paths.
    If source is a directory, a glob is run using the provided pattern.
    If source is a string containing a wildcard, glob is applied.
    Otherwise, if it's a list, it is returned directly.
    """
    if isinstance(source, list):
        return [Path(s) for s in source]
    elif Path(source).is_dir():
        patterns = ["*.png", "*.jpg", "*.jpeg"]
        return list(chain.from_iterable([Path(source).glob(p) for p in patterns]))
    elif isinstance(source, str) and '*' in source:
        return [Path(p) for p in glob(source)]
    else:
        return [source]

In [9]:
imglob = tmp_path.as_posix() +"/*.png"
images_df = daft.from_glob_path(imglob).with_column_renamed("path", "path_full_img")
images_df = images_df.with_column("image", daft.col("path_full_img"
                                 ).url.download().image.decode(
                                     mode="RGB", on_error="null")
                                 )
images_df = images_df.where(images_df["image"].not_null())

TransformImForModel = TransformImageCol.with_init_args(model_name=MODEL_NAME)

images_df = images_df.with_column("image_transformed", TransformImForModel(daft.col("image"))
                                  ).exclude("image", "num_rows")

images_df.show(1)

path_full_imgUtf8,sizeInt64,image_transformedPython
file://tmp-test-images/746.png,270443,"tensor([[[-0.2342, -0.2171, -0.3712, ..., 0.5364, 0.4851, 0.3481], [-0.4397, -0.4397, -0.5596, ..., 0.5193, 0.3823, 0.1768], [-0.7308, -0.7650, -0.8849, ..., 0.4337, 0.2111, -0.1143], ..., [ 0.6563, 0.8104, 0.9988, ..., 2.2489, 2.2489, 2.2318], [ 0.1083, 0.3138, 0.5364, ..., 2.1975, 2.1975, 2.1804], [-0.0287, 0.1768, 0.4166, ..., 2.1804, 2.1804, 2.1633]], [[-0.1099, -0.0924, -0.2500, ..., 0.6779, 0.6254, 0.4853], [-0.3200, -0.3200, -0.4426, ..., 0.6604, 0.5203, 0.3102], [-0.6176, -0.6527, -0.7752, ..., 0.5728, 0.3452, 0.0126], ..., [ 0.8004, 0.9580, 1.1506, ..., 2.4286, 2.4286, 2.4111], [ 0.2402, 0.4503, 0.6779, ..., 2.3761, 2.3761, 2.3585], [ 0.1001, 0.3102, 0.5553, ..., 2.3585, 2.3585, 2.3410]], [[ 0.1128, 0.1302, -0.0267, ..., 0.8971, 0.8448, 0.7054], [-0.0964, -0.0964, -0.2184, ..., 0.8797, 0.7402, 0.5311], [-0.3927, -0.4275, -0.5495, ..., 0.7925, 0.5659, 0.2348], ..., [ 1.0191, 1.1759, 1.3677, ..., 2.6400, 2.6400, 2.6226], [ 0.4614, 0.6705, 0.8971, ..., 2.5877, 2.5877, 2.5703], [ 0.3219, 0.5311, 0.7751, ..., 2.5703, 2.5703, 2.5529]]])"


In [ ]:
def compute_embeddings(model_name: str,
                       batch_size: int = BATCH_SIZE,
                       images_df: None | daft.DataFrame = None,
                       images_folder: None | Path = None) -> list[np.ndarray]:
    """
    Given a model name and a filelist (list of image paths), this function computes and returns a list
    of embeddings (one per image). The function instantiates an Embedder, builds a dataset and dataloader,
    and processes images in batches.
    """
    embedder = Embedder(model_name=model_name)

    if images_df is not None:
        dataset = images_df.to_torch_iter_dataset()
    elif images_folder:
        # use vanilla torch dataloader
        image_list = get_file_list(images_folder)
        dataset = ImageListIteratorAsDict(image_list, embedder.transform)


    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    embeddings = []
    for i, batch_images in enumerate(tqdm(dataloader, unit_scale=BATCH_SIZE)):
        emb = embedder.embed(batch_images["image_transformed"]).cpu().numpy()

        if i == 0:
            print(f"Shape of embedding for one batch: {emb.shape}")
        embeddings.append(emb)
    embeddings = np.vstack(embeddings)

    return embeddings




: 

In [ ]:
%%memray_flamegraph --trace-python-allocators --native --leaks --temporal --max-memory-records 10

if USE_DAFT:
    embeddings = compute_embeddings(MODEL_NAME, BATCH_SIZE, images_df=images_df)
else:
    # use vanilla torch dataset
    embeddings = compute_embeddings(MODEL_NAME, BATCH_SIZE, images_folder=IMAGES_FOLDER)

gc.collect()

Memray WARNING: Correcting symbol for malloc from 0x420620 to 0x7f7477933c60
Memray WARNING: Correcting symbol for free from 0x420ab0 to 0x7f7477934370
0it [00:00, ?it/s]

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Filter: 00:00 

🗡️ 🐟 Project: 00:00 